In [ ]:
smdg_tcl_file = '../data/raw/smdg-tcl-v20210401.csv'
    
terminals_geojson = '../data/processed/terminals.geojson'
terminals_gpkg = '../data/gpkg/terminals.gpkg'

In [ ]:
import seaconex

import pandas as pd
import numpy as np
import geopandas as gpd

from shapely.geometry import Point, LineString

## SMDG Terminal Code List (TCL)

https://smdg.org/documents/smdg-code-lists/smdg-terminal-code-list/  
https://github.com/smdg-org/Terminal-Code-List  
https://www.google.com/maps/d/viewer?mid=1nbJI9NkpAayVpSJqy13qJ2xhatcIKQJK&ll=-3.81666561775622e-14%2C-4.5703125&z=2  

In [ ]:
seaconex.tools.terminals.get_all_smdg_tcl()

In [ ]:
df_smdg_tcl = pd.read_csv(
    smdg_tcl_file,
    sep=','
).drop(
    columns=[
        'Latitude (DMS)',
        'Longitude (DMS)',
        'Last change',
        'Valid from',
        'Valid until',
    ]
).rename(
    columns={
        'UNLOCODE':'port_unlocode',
        'Alternative UNLOCODEs':'port_unlocode_alt',
        'Terminal Code': 'terminal',
        'Latitude': 'latitude',
        'Longitude': 'longitude',
        'Terminal Facility Name':'terminal_name',
        'Terminal Company Name': 'terminal_operator',
        'Terminal Website':'terminal_website',
        'Terminal Address':'terminal_address',
        'Remarks': 'remarks'
    }
).fillna("")

In [ ]:
terminals = [    
    'PSAP', 
    'NCSPA', 
    'ACOT', 
    'DPWS', 
    'RDT',
    'BCMT',
    'FIT',
    'GCMT',
    'NCMT',
    'NIT',
    'SMT',
    'DMT',
    'PSAP',
    'MAHER',
    'PNCT',
    'FCOVE',
    'CTA',
    'UCT',
    'AET',
    'K869',
    'ACOT',
    'RDT',
    'RSCT',
    'DPWLG',
    'CT1'
]

df_smdg_tcl = df_smdg_tcl[df_smdg_tcl.terminal.isin(terminals)].reset_index(drop=True)
df_smdg_tcl['code_provider'] = 'SMDG'

df_smdg_tcl.loc[df_smdg_tcl['terminal'] == 'DPWLG', 'terminal_website'] = "https://www.londongateway.com/"
df_smdg_tcl.loc[df_smdg_tcl['terminal'] == 'DPWLG', 'terminal_address'] = "1 London Gateway, Stanford-le-Hope, Essex, SS17 9DY, United Kingdom"

df_smdg_tcl.loc[df_smdg_tcl['terminal'] == 'DPWS', 'terminal_website'] = "https://www.dpworldsouthampton.com/"
df_smdg_tcl.loc[df_smdg_tcl['terminal'] == 'DPWS', 'terminal_address'] = "DP World Southampton, Western Docks, Southampton SO15 1DA, United Kingdom"

df_smdg_tcl.loc[df_smdg_tcl['terminal'] == 'CT1', 'terminal_address']= "APM Terminals Gothenburg Port Entry, Ytterhamnsvägen 1, Gothenburg, Sweden"

df_smdg_tcl.loc[df_smdg_tcl['terminal'] == 'DMT', 'terminal_address']= "2700 Broening Highway, Baltimore, MD 21222, USA"
df_smdg_tcl.loc[df_smdg_tcl['terminal'] == 'SMT', 'terminal_address']= "2600 Broening Highway, Baltimore, MD 21222, USA"

In [ ]:
df_smdg_tcl.head()

In [ ]:
df_non_smdg_tcl = pd.read_csv('../data/raw/non-smdg-terminals.csv').fillna("")

In [ ]:
df_non_smdg_tcl.head()

In [ ]:
df_non_smdg_tcl[['terminal_lat','terminal_lon']] = df_non_smdg_tcl[['terminal_lat','terminal_lon']].astype(str).apply(lambda x: x.str.replace(' ', ''))
df_non_smdg_tcl[['terminal_lat','terminal_lon']] = df_non_smdg_tcl[['terminal_lat','terminal_lon']].astype(str).apply(lambda x: x.str[1:] + x.str[0])
df_non_smdg_tcl['latitude'] = df_non_smdg_tcl['terminal_lat'].apply(lambda x: seaconex.dms_to_dd(x))
df_non_smdg_tcl['longitude'] = df_non_smdg_tcl['terminal_lon'].apply(lambda x: seaconex.dms_to_dd(x))

df_non_smdg_tcl.drop(columns=['terminal_lat','terminal_lon'], inplace=True)

df_non_smdg_tcl = df_non_smdg_tcl[df_smdg_tcl.columns.tolist()]

In [ ]:
df_non_smdg_tcl.head()

In [ ]:
df_terminals = pd.concat(
    objs=[df_non_smdg_tcl, df_smdg_tcl],
    ignore_index=True,
).sort_values(
    by=['port_unlocode']
).reset_index(drop=True)

In [ ]:
df_terminals

In [ ]:
gdf_terminals = gpd.GeoDataFrame(
      df_terminals,
      crs='EPSG:3857',
      # crs=3857,
      geometry=gpd.points_from_xy(
          df_terminals.longitude,
          df_terminals.latitude
      )
  ).drop(
      columns=[
        'latitude',
        'longitude'
      ]
  )

In [ ]:
gdf_terminals

In [ ]:
seaconex.gdf_to_geo_file(
    gdf_terminals,
    terminals_geojson,
    terminals_gpkg
)